In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
base_grande = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
base_grande.shape
base_grande = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
base_grande.shape
base_grande = base_grande.dropna()
base_grande = pd.get_dummies(base_grande, prefix = 'indicatrice_', columns=['patho_niv1'], dtype='int')

base_grande = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
base_grande.shape

liste_var = base_grande.columns[17:]
liste_bis = []
for i in liste_var : 
    liste_bis.append(i)
indicatrice_patho_niv1 = liste_bis

liste_auxil = base_grande['cla_age_5'].unique()
inter_excl = ('95et+', 'tsage', np.nan)

inter_def = []
for i in liste_auxil :
    if i in inter_excl : 
        inter_def = inter_def
    else : 
        inter_def.append(i)
classe_age = []
for i in base_grande['cla_age_5'] : 
    if i in inter_def :
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    elif i == 'tsage' : 
        age = np.nan
    elif i == '95et+' : 
        age = 100
    else : 
        age = np.nan
    classe_age.append(age)
base_grande.loc[:,'classe_age'] = classe_age




In [ ]:
"""
def creation_indicatrice (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    base_grande.loc[:,f"{nom_de_maladie}_ind"] = temporaire
"""
#utiliser get_dummies


base_grande = pd.get_dummies(base_grande, prefix = 'indicatrice_', columns=['patho_niv1'], dtype='int')

liste_var = base_grande.columns[17:]
liste_bis = []
for i in liste_var : 
    liste_bis.append(i)
indicatrice_patho_niv1 = liste_bis

liste_auxil = base_grande['cla_age_5'].unique()
inter_excl = ('95et+', 'tsage', np.nan)

inter_def = []
for i in liste_auxil :
    if i in inter_excl : 
        inter_def = inter_def
    else : 
        inter_def.append(i)
classe_age = []
for i in base_grande['cla_age_5'] : 
    if i in inter_def :
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    elif i == 'tsage' : 
        age = np.nan
    elif i == '95et+' : 
        age = 100
    else : 
        age = np.nan
    classe_age.append(age)
base_grande.loc[:,'classe_age'] = classe_age
base_grande.head()

In [ ]:
base_grande

In [ ]:
"test_rep = base_grande[['ind','dept', 'classe_age']]"
"test_rep"
#le nom de la colonne des maladies respiratoires + l'index m'empêche de créer le database
#--> ma base n'a plus les indicatrices (ancienne versions)
#pb réglé avec les dummies mais pb avec les classes d'âges mtn 

In [ ]:
model_resp_alpha = base_grande[['dept', 'classe_age','indicatrice__Maladies respiratoires chroniques (hors mucoviscidose)','sexe','region']]

In [ ]:
model_resp_alpha.columns = model_resp_alpha.columns.str.replace(' ', '_')
#model_resp_alpha = model_resp_alpha.rename(columns={'indicatrice__Maladies_respiratoires _chroniques_(hors_mucoviscidose)': 'indicatrice_Maladies_respiratoires _chroniques_(hors_mucoviscidose)'})
#model_resp_alpha = model_resp_alpha.rename(columns={'indicatrice_Maladies_respiratoires _chroniques_(hors_mucoviscidose)': 'indicatrice_Maladies_respiratoires _chroniques_'})

#j'ai renommé la colonne en ajoutant des underscores car les espaces dans son nom posaient problème 
#j'ai essayé de supp le contenu dans la parenthèse par différents moyens mais ca ne fonctionnait jamais 
#j'ai essayé aussi de supp ce double underscore au début mais ca marche pas 
#seule la premiere ligne de la cellule abouti 
#pb je peux pas groupe by pour faire mon histogramme par age 

In [ ]:
model_resp_alpha

In [ ]:
resultats = model_resp_alpha[model_resp_alpha['indicatrice__Maladies_respiratoires_chroniques_(hors_mucoviscidose)'] == 1].groupby('classe_age').size()

#les résultats n'ont aucun sens 